In [0]:
run Lattice_functions.ipynb

# Model 1: Constant Stiffness Model : Force variation

In [0]:
#Generating the lattice
nx = 20 + 1 # 1 added to make the total length = 20*0.01
ny = 20 + 1
lx = 0.05
ly = 0.05
k_inPlane = 7e4
nz = 3
lz = 0.01
k_vertical = 1e8 # to enzure strain in z direction is close to zero
M1 = Rect_Lattice(nx, ny, lx, ly, k_inPlane, [], [], [])
M1 = add_layers_in_z_direction(M1, nz, lz, k_vertical, 0, [])

#setting z of the middle layer to zero and deriving support criterias:
min_x = M1.nodes[1].x
min_y = M1.nodes[1].y
max_x = M1.nodes[1].x
max_y = M1.nodes[1].y

for i in M1.nodes:
    i.z = round(i.z, 12) - lz
    i.x = round(i.x, 12)
    i.y = round(i.y, 12)
    min_x = min(i.x, min_x)
    min_y = min(i.y, min_y)
    max_x = max(i.x, max_x)
    max_y = max(i.y, max_y)

#Adding support
for i in M1.nodes:
    if(i.z == 0):
        if(i.x == min_x or i.x == max_x or i.y == min_y or i.y == max_y):
            i.s_x = 1
            i.s_y = 1
            i.s_z = 1
            M1.support_x.append(i.n)
            M1.support_y.append(i.n)
            M1.support_z.append(i.n)


mid_node = int(nx*ny + (nx*ny + 1)/2) #Only for odd nx and ny

In [0]:
#Optimization preparation:
x0 = []
y0 = []
z0 = []
for i in M1.nodes:
    x0.insert((i.n - 1), i.x)
    y0.insert((i.n - 1), i.y)
    z0.insert((i.n - 1), i.z)
    
coord0 = x0 + y0 + z0
nNodes = int(len(coord0)/3)

#bounds:
lb = -np.inf*np.ones(len(coord0))
ub = np.inf*np.ones(len(coord0))

for i in M1.support_x:
    lb[i - 1] = x0[i - 1]
    ub[i - 1] = x0[i - 1]

for i in M1.support_y:
    lb[nNodes + i - 1] = y0[i - 1]
    ub[nNodes + i - 1] = y0[i - 1]
    
for i in M1.support_z:
    lb[(2*nNodes) + i - 1] = z0[i - 1]
    ub[(2*nNodes) + i - 1] = z0[i - 1]


bounds = Bounds(lb, ub)

#Defining half unit forces
f0 = Point_load_at_centre(0.5, M1)

#Variable for saving the data
data_M1 = [] #has the coordinates of the energy minimized lattices
forces_M1 = [] #forces applied at the centre
energies_M1 = [] #energies of the optimized lattices
z_centre_M1 = [] #deflection at centre
results_M1 = [] #Outputs from scipy.optimize

In [0]:
for i in range(1, 22):
  print(i)
  f = (i - 1)*f0
  result = minimize(Energy, coord0, args = (f, M1), method='L-BFGS-B', jac = Jacobian, options={'iprint': 1}, bounds=bounds)
  res = result.x
  res.tolist()
  data_M1.append(res)
  forces_M1.append(max(f))
  energies_M1.append(result.fun)
  z_centre_M1.append(res[2*nNodes + mid_node - 1])
  results_M1.append(result)

In [0]:
from numpy import save
save('data_M1.npy', data_M1)
save('forces_M1.npy', forces_M1)
save('energies_M1.npy', energies_M1)
save('z_centre_M1.npy', z_centre_M1)
save('results_M1.npy', results_M1)
save('M1.npy', M1)

# Model 1: Constant AE Model : Force variation

In [0]:
#Generating the lattice
nx = 20 + 1 # 1 added to make the total length = 100*0.01
ny = 20 + 1
lx = 0.05
ly = 0.05
k_inPlane = 10
nz = 3
lz = 0.01
k_vertical = 10
M1_AE = Rect_Lattice(nx, ny, lx, ly, k_inPlane, [], [], [])
M1_AE = add_layers_in_z_direction(M1_AE, nz, lz, k_vertical, 0, [])

# modifying stiffness wrt AE = constant
AE = 4e3
for i in M1_AE.springs:
  i.k = AE/(i.l0)


for i in M1_AE.springs:
  if (abs(M1_AE.nodes[i.i - 1].z - M1_AE.nodes[i.j - 1].z) != 0):
    i.k = i.k*1e2


#setting z of the middle layer to zero and deriving support criterias:
min_x = M1_AE.nodes[1].x
min_y = M1_AE.nodes[1].y
max_x = M1_AE.nodes[1].x
max_y = M1_AE.nodes[1].y

for i in M1_AE.nodes:
    i.z = round(i.z, 12) - lz
    i.x = round(i.x, 12)
    i.y = round(i.y, 12)
    min_x = min(i.x, min_x)
    min_y = min(i.y, min_y)
    max_x = max(i.x, max_x)
    max_y = max(i.y, max_y)

#Adding support
for i in M1_AE.nodes:
    if(i.z == 0):
        if(i.x == min_x or i.x == max_x or i.y == min_y or i.y == max_y):
            i.s_x = 1
            i.s_y = 1
            i.s_z = 1
            M1_AE.support_x.append(i.n)
            M1_AE.support_y.append(i.n)
            M1_AE.support_z.append(i.n)


mid_node = int(nx*ny + (nx*ny + 1)/2) #Only for odd nx and ny

In [0]:
#Optimization preparation:
x0 = []
y0 = []
z0 = []
for i in M1_AE.nodes:
    x0.insert((i.n - 1), i.x)
    y0.insert((i.n - 1), i.y)
    z0.insert((i.n - 1), i.z)
    
coord0 = x0 + y0 + z0
nNodes = int(len(coord0)/3)

#bounds:
lb = -np.inf*np.ones(len(coord0))
ub = np.inf*np.ones(len(coord0))

for i in M1_AE.support_x:
    lb[i - 1] = x0[i - 1]
    ub[i - 1] = x0[i - 1]

for i in M1_AE.support_y:
    lb[nNodes + i - 1] = y0[i - 1]
    ub[nNodes + i - 1] = y0[i - 1]
    
for i in M1_AE.support_z:
    lb[(2*nNodes) + i - 1] = z0[i - 1]
    ub[(2*nNodes) + i - 1] = z0[i - 1]


bounds = Bounds(lb, ub)

#Defining half unit forces
f0 = Point_load_at_centre(0.5, M1_AE)

#Variable for saving the data
data_M1_AE = [] #has the coordinates of the energy minimized lattices
forces_M1_AE = [] #forces applied at the centre
energies_M1_AE = [] #energies of the optimized lattices
z_centre_M1_AE = [] #deflection at centre
results_M1_AE = [] #Outputs from scipy.optimize

In [0]:
for i in range(1, 22):
  print(i)
  f = (i - 1)*f0
  result = minimize(Energy, coord0, args = (f, M1_AE), method='L-BFGS-B', jac = Jacobian, options={'iprint': 1}, bounds=bounds)
  res = result.x
  res.tolist()
  data_M1_AE.append(res)
  forces_M1_AE.append(max(f))
  energies_M1_AE.append(result.fun)
  z_centre_M1_AE.append(res[2*nNodes + mid_node - 1])
  results_M1_AE.append(result)

In [0]:
from numpy import save
save('data_M1_AE.npy', data_M1_AE)
save('forces_M1_AE.npy', forces_M1_AE)
save('energies_M1_AE.npy', energies_M1_AE)
save('z_centre_M1_AE.npy', z_centre_M1_AE)
save('results_M1_AE.npy', results_M1_AE)
save('M1_AE.npy', M1_AE)

# Model 1: Constant Stiffness Model : Thickness variation

In [0]:
#Variable for saving the data
data_M1_t = [] #has the coordinates of the energy minimized lattices
thickness_M1_t = [] #Thickness of the model
energies_M1_t = [] #energies of the optimized lattices
z_centre_M1_t = [] #deflection at centre
results_M1_t = [] #Outputs from scipy.optimize
lat_M1_t = [] #Lattices with varying thickness

In [0]:
#Model 1, thickness varying condition
for i in range(1, 11):
  print(i)
  thickness = 0.005*i

  #Generating the lattice
  nx = 20 + 1 # 1 added to make the total length = 20*0.01
  ny = 20 + 1
  lx = 0.05
  ly = 0.05
  k_inPlane = 7e4
  nz = 3
  lz = thickness
  k_vertical = 1e8 # to enzure strain in z direction is close to zero

  M1_t = Rect_Lattice(nx, ny, lx, ly, k_inPlane, [], [], [])
  M1_t = add_layers_in_z_direction(M1_t, nz, lz, k_vertical, 0, [])

  #setting z of the middle layer to zero:
  min_x = M1_t.nodes[1].x
  min_y = M1_t.nodes[1].y
  max_x = M1_t.nodes[1].x
  max_y = M1_t.nodes[1].y

  for i in M1_t.nodes:
    i.z = round(i.z, 12) - lz
    i.x = round(i.x, 12)
    i.y = round(i.y, 12)
    min_x = min(i.x, min_x)
    min_y = min(i.y, min_y)
    max_x = max(i.x, max_x)
    max_y = max(i.y, max_y)

  #Adding support
  for i in M1_t.nodes:
    if(i.z == 0):
      if(i.x == min_x or i.x == max_x or i.y == min_y or i.y == max_y):
        i.s_x = 1
        i.s_y = 1
        i.s_z = 1
        M1_t.support_x.append(i.n)
        M1_t.support_y.append(i.n)
        M1_t.support_z.append(i.n)


  mid_node = int(nx*ny + (nx*ny + 1)/2) #Only for odd nx and ny


  #Optimization preparation:
  x0 = []
  y0 = []
  z0 = []
  for i in M1_t.nodes:
    x0.insert((i.n - 1), i.x)
    y0.insert((i.n - 1), i.y)
    z0.insert((i.n - 1), i.z)
    
  coord0 = x0 + y0 + z0
  nNodes = int(len(coord0)/3)

  #bounds:
  lb = -np.inf*np.ones(len(coord0))
  ub = np.inf*np.ones(len(coord0))

  for i in M1_t.support_x:
    lb[i - 1] = x0[i - 1]
    ub[i - 1] = x0[i - 1]

  for i in M1_t.support_y:
    lb[nNodes + i - 1] = y0[i - 1]
    ub[nNodes + i - 1] = y0[i - 1]
    
  for i in M1_t.support_z:
    lb[(2*nNodes) + i - 1] = z0[i - 1]
    ub[(2*nNodes) + i - 1] = z0[i - 1]


  bounds = Bounds(lb, ub)

  #Defining 2kN forces
  f = Point_load_at_centre(2, M1_t)

  result = minimize(Energy, coord0, args = (f, M1_t), method='L-BFGS-B', jac = Jacobian, options={'iprint': 1}, bounds=bounds)
  res = result.x
  res.tolist()
  data_M1_t.append(res)
  thickness_M1_t.append(thickness)
  energies_M1_t.append(result.fun)
  z_centre_M1_t.append(res[2*nNodes + mid_node - 1])
  results_M1_t.append(result)
  lat_M1_t.append(M1_t)

In [0]:
from numpy import save
save('data_M1_t.npy', data_M1_t)
save('thickness_M1_t.npy', thickness_M1_t)
save('energies_M1_t.npy', energies_M1_t)
save('z_centre_M1_t.npy', z_centre_M1_t)
save('results_M1_t.npy', results_M1_t)
save('lat_M1_t.npy', lat_M1_t)